In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('log_reg').getOrCreate()

Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true


23/04/03 12:32:25 WARN Utils: Your hostname, root resolves to a loopback address: 127.0.1.1; using 192.168.90.240 instead (on interface wlan0)
23/04/03 12:32:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 12:32:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/03 12:32:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/03 12:32:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df=spark.read.csv('Log_Reg_dataset.csv',inferSchema=True,header=True)

In [3]:
print((df.count(), len(df.columns)))

(20000, 6)


In [4]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



In [5]:
df.show(5)

+---------+---+--------------+--------+----------------+------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
|    India| 41|             1|   Yahoo|              21|     1|
|   Brazil| 28|             1|   Yahoo|               5|     0|
|   Brazil| 40|             0|  Google|               3|     0|
|Indonesia| 31|             1|    Bing|              15|     1|
| Malaysia| 32|             0|  Google|              15|     1|
+---------+---+--------------+--------+----------------+------+
only showing top 5 rows



In [6]:
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



In [7]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



In [9]:
df.groupBy('Platform').count().show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



In [10]:
df.groupBy('Country').mean().show()

+---------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+-------------------+---------------------+--------------------+
| Malaysia|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+-------------------+---------------------+--------------------+



In [11]:
df.groupBy('Platform').mean().show()

+--------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+-------------------+---------------------+------------------+
|   Yahoo|28.569226087838523| 0.5094837204584644|    9.599655137437875|0.5071508266558474|
|    Bing| 28.68394495412844| 0.4720183486238532|    9.114908256880733|0.4559633027522936|
|  Google|28.380038055699707| 0.5149628092025601|    9.804878048780488|0.5210171250648676|
+--------+------------------+-------------------+---------------------+------------------+



In [12]:
df.groupBy('Status').mean().show()

+------+--------+-------------------+---------------------+-----------+
|Status|avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|avg(Status)|
+------+--------+-------------------+---------------------+-----------+
|     1| 26.5435|             0.7019|              14.5617|        1.0|
|     0| 30.5356|             0.3039|               4.5449|        0.0|
+------+--------+-------------------+---------------------+-----------+



In [13]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [14]:
platform_indexer = StringIndexer(inputCol="Platform", outputCol ="Platform_Num").fit(df)
df = platform_indexer.transform(df)
df.show(3,False)

+-------+---+--------------+--------+----------------+------+------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_Num|
+-------+---+--------------+--------+----------------+------+------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0         |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0         |
|Brazil |40 |0             |Google  |3               |0     |1.0         |
+-------+---+--------------+--------+----------------+------+------------+
only showing top 3 rows



In [15]:
df.groupBy('Platform').count().orderBy('count', ascending=False).show(5,False)

+--------+-----+
|Platform|count|
+--------+-----+
|Yahoo   |9859 |
|Google  |5781 |
|Bing    |4360 |
+--------+-----+



In [18]:
from pyspark.ml.feature import OneHotEncoder

In [23]:
platform_encoder = OneHotEncoder(inputCol = 'Platform_Num', outputCol = "Platform_Vector")
platform_encoder.setDropLast(False)
platform_encoder = platform_encoder.fit(df)
df = platform_encoder.transform(df)
df.show(3, False)

+-------+---+--------------+--------+----------------+------+------------+---------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_Num|Platform_Vector|
+-------+---+--------------+--------+----------------+------+------------+---------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0         |(3,[0],[1.0])  |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0         |(3,[0],[1.0])  |
|Brazil |40 |0             |Google  |3               |0     |1.0         |(3,[1],[1.0])  |
+-------+---+--------------+--------+----------------+------+------------+---------------+
only showing top 3 rows



In [25]:
df.groupBy('Platform_Vector').count().orderBy('count', ascending=False).show(5, False)

+---------------+-----+
|Platform_Vector|count|
+---------------+-----+
|(3,[0],[1.0])  |9859 |
|(3,[1],[1.0])  |5781 |
|(3,[2],[1.0])  |4360 |
+---------------+-----+



In [26]:
country_indexer = StringIndexer(inputCol="Country", outputCol="Country_Num").fit(df)
df = country_indexer.transform(df)
df.groupBy('Country').count().orderBy('count', ascending=False).show(5, False)


+---------+-----+
|Country  |count|
+---------+-----+
|Indonesia|12178|
|India    |4018 |
|Brazil   |2586 |
|Malaysia |1218 |
+---------+-----+



In [27]:
df.groupBy('Country_Num').count().orderBy('count', ascending=False).show(5, False)

+-----------+-----+
|Country_Num|count|
+-----------+-----+
|0.0        |12178|
|1.0        |4018 |
|2.0        |2586 |
|3.0        |1218 |
+-----------+-----+



In [29]:
country_encoder = OneHotEncoder(inputCol="Country_Num", outputCol="Country_Vector")
country_encoder.setDropLast(False)
country_encoder = country_encoder.fit(df)
df = country_encoder.transform(df)
df.select(['Country','Country_Num','Country_Vector']).show(3, False)


+-------+-----------+--------------+
|Country|Country_Num|Country_Vector|
+-------+-----------+--------------+
|India  |1.0        |(4,[1],[1.0]) |
|Brazil |2.0        |(4,[2],[1.0]) |
|Brazil |2.0        |(4,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows



In [30]:
df.groupBy('Country_Vector').count().orderBy('count', ascending=False).show(5, False)

+--------------+-----+
|Country_Vector|count|
+--------------+-----+
|(4,[0],[1.0]) |12178|
|(4,[1],[1.0]) |4018 |
|(4,[2],[1.0]) |2586 |
|(4,[3],[1.0]) |1218 |
+--------------+-----+



In [31]:
df_assembler = VectorAssembler(inputCols=['Platform_Vector','Country_Vector','Age','Repeat_Visitor', 'Web_pages_viewed'], outputCol="features")

In [32]:
df = df_assembler.transform(df)
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Platform_Num: double (nullable = false)
 |-- Platform_Vector: vector (nullable = true)
 |-- Country_Num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
df.select(['features','Status']).show(10,False)

+----------------------------------------+------+
|features                                |Status|
+----------------------------------------+------+
|(10,[0,4,7,8,9],[1.0,1.0,41.0,1.0,21.0])|1     |
|(10,[0,5,7,8,9],[1.0,1.0,28.0,1.0,5.0]) |0     |
|(10,[1,5,7,9],[1.0,1.0,40.0,3.0])       |0     |
|(10,[2,3,7,8,9],[1.0,1.0,31.0,1.0,15.0])|1     |
|(10,[1,6,7,9],[1.0,1.0,32.0,15.0])      |1     |
|(10,[1,5,7,9],[1.0,1.0,32.0,3.0])       |0     |
|(10,[1,5,7,9],[1.0,1.0,32.0,6.0])       |0     |
|(10,[1,3,7,9],[1.0,1.0,27.0,9.0])       |0     |
|(10,[0,3,7,9],[1.0,1.0,32.0,2.0])       |0     |
|(10,[2,3,7,8,9],[1.0,1.0,31.0,1.0,16.0])|1     |
+----------------------------------------+------+
only showing top 10 rows



Let us select only features column as input and the Status column as
output for training the logistic regression model.

-----------------------------------------
Splitting the Dataset

We can print the shape of train and test data to validate the size

In [35]:
model_df=df.select(['features','Status'])

In [36]:
training_df, test_df = model_df.randomSplit([0.75, 0.25])
print(training_df.count())

14950


In [37]:
training_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 7553|
|     0| 7397|
+------+-----+



In [38]:
print(test_df.count())

5050


In [39]:
test_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 2447|
|     0| 2603|
+------+-----+



------------------------------------------------
Build and Train Logistic Regression Model

In [40]:
from pyspark.ml.classification import LogisticRegression

In [42]:
log_reg = LogisticRegression(labelCol='Status').fit(training_df)

In [43]:
train_results = log_reg.evaluate(training_df).predictions

In [46]:
train_results.filter(train_results['Status'] == 1).filter(train_results['prediction'] == 1).select(['Status','prediction','probability']).show(10,False)

+------+----------+----------------------------------------+
|Status|prediction|probability                             |
+------+----------+----------------------------------------+
|1     |1.0       |[0.42059647262953725,0.5794035273704627]|
|1     |1.0       |[0.42059647262953725,0.5794035273704627]|
|1     |1.0       |[0.2575560791410535,0.7424439208589465] |
|1     |1.0       |[0.2575560791410535,0.7424439208589465] |
|1     |1.0       |[0.2575560791410535,0.7424439208589465] |
|1     |1.0       |[0.1422051267994548,0.8577948732005451] |
|1     |1.0       |[0.07340809714391215,0.9265919028560878]|
|1     |1.0       |[0.07340809714391215,0.9265919028560878]|
|1     |1.0       |[0.07340809714391215,0.9265919028560878]|
|1     |1.0       |[0.07340809714391215,0.9265919028560878]|
+------+----------+----------------------------------------+
only showing top 10 rows



---------------------------------------------------
Evaluate Linear Regression Model on Test Data

In [47]:
results = log_reg.evaluate(test_df).predictions

In [48]:
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Status: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [49]:
results.select(['Status','prediction']).show(10,False)

+------+----------+
|Status|prediction|
+------+----------+
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |1.0       |
|1     |1.0       |
|1     |1.0       |
+------+----------+
only showing top 10 rows



----------------------------------
Confusion Matrix

In [51]:
results[(results.Status == 1) & (results.prediction == 1)].count()

2329

In [56]:
results[(results.Status == 0)& (results.prediction == 0)].count()

2445

In [57]:
results[(results.Status == 1)& (results.prediction == 0)].count()

118

In [58]:
results[(results.Status == 0)& (results.prediction == 1)].count()

158

--------------------
some formulas
accuracy=float((true_postives+true_negatives) /(results.count()))
recall = float(true_postives)/(true_postives + false_negatives)
precision = float(true_postives) / (true_postives + false_positives)